# 🧪 Test de Flujo Completo de Grafos

Este notebook valida el flujo end-to-end de todos los grafos del sistema:

## Grafos a Probar:
1. **TALENT** - Actores, Directores, Colaboraciones
2. **CONTENT** - Metadata, Discovery
3. **PLATFORM** - Availability, Presence
4. **BUSINESS** - Pricing, Rankings, Intelligence
5. **COMMON** - Validation, Admin

## Flujo de Cada Test:
```
Pregunta del Usuario
    ↓
Main Router → Selecciona Grafo
    ↓
Validation Preprocessor → Valida entidades (si es necesario)
    ↓
Sub-Grafo → Procesa pregunta
    ↓
Classifier → Selecciona nodo
    ↓
Router → Selecciona tool
    ↓
Agent → Ejecuta tool y genera respuesta
    ↓
Supervisor → Decide si continuar o terminar
    ↓
Formatter → Formatea respuesta final
```

In [1]:
# Setup
import sys
import asyncio
import json
from pprint import pprint

# Import main router
from src.strands.main_router.graph import process_question_main

print("✅ Imports completados")

Loading validation data from platform_name_iso
✅ Imports completados


## Helper Functions

In [2]:
async def test_question(question: str):
    """
    Prueba una pregunta y valida el flujo completo.
    """
    print("\n" + "="*80)
    print("="*80)
    print(f"❓ Pregunta: {question}")
    print("="*80 + "\n")
    
    try:
        result = await process_question_main(question)
        
        print("\n" + "="*80)
        print("📊 RESULTADO")
        print("="*80)
        
        # Verificar si necesita desambiguación
        if result.get('needs_user_input'):
            print("⚠️  DESAMBIGUACIÓN REQUERIDA")
            print(f"📝 Mensaje: {result.get('validation_message', 'N/A')[:300]}...")
            return {
                "status": "disambiguation",
                "question": question,
                "message": result.get('validation_message')
            }
        
        # Verificar respuesta
        answer = result.get('answer', 'N/A')
        print(f"✅ Respuesta: {answer[:400]}...")
        
        # Metadata
        print(f"\n📋 Metadata:")
        print(f"   • Grafo usado: {result.get('graph_used', 'N/A')}")
        print(f"   • Validación realizada: {result.get('validation_done', False)}")
        print(f"   • Tool calls: {result.get('tool_calls_count', 0)}")
        print(f"   • Re-routings: {result.get('re_routing_count', 0)}")
        
        print("="*80 + "\n")
        
        return {
            "status": "success",
            "question": question,
            "answer": answer[:200]
        }
        
    except Exception as e:
        print(f"\n❌ ERROR: {e}\n")
        import traceback
        traceback.print_exc()
        return {
            "status": "error",
            "question": question,
            "error": str(e)
        }

## 1. TALENT Graph Tests

### Flujo esperado:
```
Main Router → TALENT
    ↓
Validation → validate_director/validate_actor
    ↓
Talent Classifier → directors/actors/collaborations
    ↓
Router → Selecciona tool específica
    ↓
Agent → Ejecuta y responde
```

## validado


In [3]:
# Test 1.1: Director Filmography
result_1_1 = await test_question(
    question="¿Qué películas ha dirigido Christopher Nolan?"
)


❓ Pregunta: ¿Qué películas ha dirigido Christopher Nolan?

⚠️  Creando grafo con flujo SECUENCIAL (legacy)

🎯 MAIN GRAPH ROUTER
📝 Pregunta: ¿Qué películas ha dirigido Christopher Nolan?
TALENT[ROUTER] Decisión raw: {'ROLE': 'ASSISTANT', 'CONTENT': [{'TEXT': 'TALENT'}]}
[ROUTER] ✅ Grafo seleccionado: talent


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: ¿Qué películas ha dirigido Christopher Nolan?
[VALIDATION] 🤖 Ejecutando validación con LLM...
Para comenzar, validaré el nombre del director:
Tool #1: validate_director

🔍 SQL QUERY EJECUTADA
📝 Operación: director exact search
📄 Query:

WITH q AS (SELECT %s::text AS s)
SELECT 
  d.id, 
  d.name,
  t.n_titles
FROM ms.directors d
CROSS JOIN q
LEFT JOIN LATERAL (
  SELECT COUNT(*)::integer AS n_titles
  FROM ms.directed_by db 
  WHERE db.director_id = d.id
) t ON TRUE
WHERE d.name ILIKE q.s
ORDER BY t.n_titles DESC NULLS LAST, d.name ASC
LIMIT 25

🔧 Parámetros: ('Christopher Nolan',)

✅ Query retornó 1 filas en 1.418s

Christopher Nolan validado 

## no validado

In [3]:
# Test 1.2: Director Collaborators
result_1_2 = await test_question(
    question="¿Con quién ha colaborado Steven Spielberg?",
)



❓ Pregunta: ¿Con quién ha colaborado Steven Spielberg?

⚠️  Creando grafo con flujo SECUENCIAL (legacy)

🎯 MAIN GRAPH ROUTER
📝 Pregunta: ¿Con quién ha colaborado Steven Spielberg?
TALENT[ROUTER] Decisión raw: {'ROLE': 'ASSISTANT', 'CONTENT': [{'TEXT': 'TALENT'}]}
[ROUTER] ✅ Grafo seleccionado: talent


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: ¿Con quién ha colaborado Steven Spielberg?
[VALIDATION] 🤖 Ejecutando validación con LLM...
NO_VALIDATION_NEEDED

Entiendo que quieres información sobre las colaboraciones de Steven Spielberg, pero la herramienta que tengo disponible solo permite validar su nombre como director. Primero, validaré su nombre:
Tool #1: validate_director

🔍 SQL QUERY EJECUTADA
📝 Operación: director exact search
📄 Query:

WITH q AS (SELECT %s::text AS s)
SELECT 
  d.id, 
  d.name,
  t.n_titles
FROM ms.directors d
CROSS JOIN q
LEFT JOIN LATERAL (
  SELECT COUNT(*)::integer AS n_titles
  FROM ms.directed_by db 
  WHERE db.director_id = d.id
) t ON TRUE
WHERE d.name ILIKE q.

In [4]:
# Test 1.3: Actor Filmography
result_1_3 = await test_question(
    question="¿En qué películas ha actuado Tom Hanks?"
)


❓ Pregunta: ¿En qué películas ha actuado Tom Hanks?

⚠️  Creando grafo con flujo SECUENCIAL (legacy)

🎯 MAIN GRAPH ROUTER
📝 Pregunta: ¿En qué películas ha actuado Tom Hanks?
TALENT[ROUTER] Decisión raw: {'ROLE': 'ASSISTANT', 'CONTENT': [{'TEXT': 'TALENT'}]}
[ROUTER] ✅ Grafo seleccionado: talent


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: ¿En qué películas ha actuado Tom Hanks?
[VALIDATION] 🤖 Ejecutando validación con LLM...
NO_VALIDATION_NEEDED

Primero voy a validar el actor para asegurar su identificación única:
Tool #1: validate_actor

🔍 SQL QUERY EJECUTADA
📝 Operación: actor exact search
📄 Query:

SELECT id, name
FROM ms.cast
WHERE name ILIKE %s               
ORDER BY name ASC
LIMIT 25

🔧 Parámetros: ('Tom Hanks',)

✅ Query retornó 1 filas en 1.893s

Tom Hanks validado (ID: 805619)

Sin embargo, para listar todas sus películas necesitaría una herramienta adicional que no está disponible en este conjunto de funciones. La función actual solo valida la existencia del actor, pero no recu

In [5]:
# Test 1.4: Actor Coactors
result_1_4 = await test_question(
    question="¿Con quién ha actuado Brad Pitt frecuentemente?"
)


❓ Pregunta: ¿Con quién ha actuado Brad Pitt frecuentemente?

⚠️  Creando grafo con flujo SECUENCIAL (legacy)

🎯 MAIN GRAPH ROUTER
📝 Pregunta: ¿Con quién ha actuado Brad Pitt frecuentemente?
TALENT[ROUTER] Decisión raw: {'ROLE': 'ASSISTANT', 'CONTENT': [{'TEXT': 'TALENT'}]}
[ROUTER] ✅ Grafo seleccionado: talent


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: ¿Con quién ha actuado Brad Pitt frecuentemente?
[VALIDATION] 🤖 Ejecutando validación con LLM...
NO_VALIDATION_NEEDED

La pregunta requeriría un análisis de la carrera de Brad Pitt que va más allá de una simple validación de entidad. Para responder a esta pregunta necesitaríamos herramientas adicionales de búsqueda de información sobre filmografía y colaboraciones, que no están disponibles en el conjunto de herramientas actual.

Sin embargo, si deseas que valide el nombre de Brad Pitt como actor, puedo hacerlo:
Tool #1: validate_actor

🔍 SQL QUERY EJECUTADA
📝 Operación: actor exact search
📄 Query:

SELECT id, name
FROM ms.cast
WHERE name IL

In [ ]:
# Test 1.5: Actor-Director Collaborations
result_1_5 = await test_question(
    question="¿En qué películas han trabajado juntos Leonardo DiCaprio y Martin Scorsese?",
    expected_graph="talent",
    description="Actor-Director Collaborations"
)

## 2. CONTENT Graph Tests

### Flujo esperado:
```
Main Router → CONTENT
    ↓
Validation → validate_title (si menciona título específico)
    ↓
Content Classifier → metadata/discovery
    ↓
Router → Selecciona tool específica
    ↓
Agent → Ejecuta y responde
```

In [ ]:
# Test 2.1: Metadata Query - Year
result_2_1 = await test_question(
    question="¿De qué año es Inception?",
    expected_graph="content",
    description="Metadata Query - Year of Inception"
)

In [ ]:
# Test 2.2: Metadata Count
result_2_2 = await test_question(
    question="¿Cuántas películas de acción hay en el catálogo?",
    expected_graph="content",
    description="Metadata Count - Action Movies"
)

In [ ]:
# Test 2.3: Metadata List
result_2_3 = await test_question(
    question="¿Qué géneros están disponibles?",
    expected_graph="content",
    description="Metadata List - Available Genres"
)

In [ ]:
# Test 2.4: Metadata Stats
result_2_4 = await test_question(
    question="Dame estadísticas del catálogo de películas",
    expected_graph="content",
    description="Metadata Stats - Catalog Statistics"
)

In [ ]:
# Test 2.5: Discovery - Filmography by UID
result_2_5 = await test_question(
    question="Dame información completa de The Matrix",
    expected_graph="content",
    description="Discovery - Complete Info"
)

## 3. PLATFORM Graph Tests

### Flujo esperado:
```
Main Router → PLATFORM
    ↓
Validation → validate_title (para availability)
    ↓
Platform Classifier → availability/presence
    ↓
Router → Selecciona tool específica
    ↓
Agent → Ejecuta y responde
```

In [ ]:
# Test 3.1: Availability by Title
result_3_1 = await test_question(
    question="¿Dónde puedo ver The Matrix?",
    expected_graph="platform",
    description="Availability - The Matrix"
)

In [ ]:
# Test 3.2: Platform Exclusives
result_3_2 = await test_question(
    question="¿Qué contenido exclusivo tiene Netflix en Estados Unidos?",
    expected_graph="platform",
    description="Platform Exclusives - Netflix US"
)

In [ ]:
# Test 3.3: Presence Count
result_3_3 = await test_question(
    question="¿Cuántos títulos tiene Disney+ en Argentina?",
    expected_graph="platform",
    description="Presence Count - Disney+ Argentina"
)

In [ ]:
# Test 3.4: Presence Statistics
result_3_4 = await test_question(
    question="Dame estadísticas de contenido en plataformas de streaming",
    expected_graph="platform",
    description="Presence Statistics - Overall"
)

In [ ]:
# Test 3.5: Recent Premieres
result_3_5 = await test_question(
    question="¿Qué estrenos recientes hay en México?",
    expected_graph="platform",
    description="Recent Premieres - Mexico"
)

## 4. BUSINESS Graph Tests

### Flujo esperado:
```
Main Router → BUSINESS
    ↓
Business Classifier → pricing/rankings/intelligence
    ↓
Router → Selecciona tool específica
    ↓
Agent → Ejecuta y responde
```

In [ ]:
# Test 4.1: Pricing - Latest
result_4_1 = await test_question(
    question="¿Cuál es el precio actual de Netflix?",
    expected_graph="business",
    description="Pricing - Netflix Latest"
)

In [ ]:
# Test 4.2: Pricing - Historical
result_4_2 = await test_question(
    question="¿Cómo ha evolucionado el precio de Disney+ en el último año?",
    expected_graph="business",
    description="Pricing - Disney+ Historical"
)

In [ ]:
# Test 4.3: Rankings - Top Generic
result_4_3 = await test_question(
    question="¿Cuáles son las películas más populares?",
    expected_graph="business",
    description="Rankings - Top Movies"
)

In [ ]:
# Test 4.4: Intelligence - Catalog Similarity
result_4_4 = await test_question(
    question="¿Qué tan similar es el catálogo de Netflix y HBO Max?",
    expected_graph="business",
    description="Intelligence - Catalog Similarity"
)

## 5. Desambiguación Tests

### Flujo esperado:
```
Main Router → Grafo
    ↓
Validation → Detecta múltiples coincidencias
    ↓
Sistema retorna needs_user_input = True
    ↓
Usuario elige opción
    ↓
Segunda llamada con resolved_entity
    ↓
Continúa flujo normal
```

In [ ]:
# Test 5.1: Título ambiguo
result_5_1 = await test_question(
    question="¿De qué año es The Matrix?",
    expected_graph="content",
    description="Disambiguation - The Matrix (múltiples versiones)"
)

In [ ]:
# Test 5.2: Actor ambiguo (si existe)
result_5_2 = await test_question(
    question="¿Qué películas ha hecho Michael Jordan?",
    expected_graph="talent",
    description="Disambiguation - Michael Jordan (actor vs deportista)"
)

## 📊 Resumen de Resultados

In [ ]:
# Recopilar todos los resultados
all_results = [
    result_1_1, result_1_2, result_1_3, result_1_4, result_1_5,  # TALENT
    result_2_1, result_2_2, result_2_3, result_2_4, result_2_5,  # CONTENT
    result_3_1, result_3_2, result_3_3, result_3_4, result_3_5,  # PLATFORM
    result_4_1, result_4_2, result_4_3, result_4_4,              # BUSINESS
    result_5_1, result_5_2                                        # DISAMBIGUATION
]

# Contar por status
success_count = sum(1 for r in all_results if r['status'] == 'success')
error_count = sum(1 for r in all_results if r['status'] == 'error')
disambiguation_count = sum(1 for r in all_results if r['status'] == 'disambiguation')
total = len(all_results)

print("\n" + "="*80)
print("📊 RESUMEN FINAL DE TESTS")
print("="*80)
print(f"\n✅ Exitosos: {success_count}/{total} ({success_count/total*100:.1f}%)")
print(f"⚠️  Desambiguación: {disambiguation_count}/{total} ({disambiguation_count/total*100:.1f}%)")
print(f"❌ Errores: {error_count}/{total} ({error_count/total*100:.1f}%)")

print("\n" + "-"*80)
print("📋 Detalle por Grafo:")
print("-"*80)

# Agrupar por grafo
by_graph = {}
for r in all_results:
    graph = r['expected_graph']
    if graph not in by_graph:
        by_graph[graph] = {'success': 0, 'error': 0, 'disambiguation': 0, 'total': 0}
    by_graph[graph][r['status']] += 1
    by_graph[graph]['total'] += 1

for graph, stats in by_graph.items():
    success_pct = stats['success'] / stats['total'] * 100
    print(f"\n🔹 {graph.upper()}:")
    print(f"   ✅ {stats['success']}/{stats['total']} exitosos ({success_pct:.1f}%)")
    if stats['disambiguation'] > 0:
        print(f"   ⚠️  {stats['disambiguation']} con desambiguación")
    if stats['error'] > 0:
        print(f"   ❌ {stats['error']} con errores")

print("\n" + "="*80 + "\n")

## 🔍 Análisis de Errores

In [ ]:
# Mostrar detalles de errores
errors = [r for r in all_results if r['status'] == 'error']

if errors:
    print("\n" + "="*80)
    print("❌ ERRORES ENCONTRADOS")
    print("="*80)
    
    for i, err in enumerate(errors, 1):
        print(f"\n{i}. {err['question']}")
        print(f"   Grafo esperado: {err['expected_graph']}")
        print(f"   Error: {err['error']}")
else:
    print("\n✅ No se encontraron errores")

## 📝 Exportar Resultados

In [ ]:
# Guardar resultados en JSON
import json
from datetime import datetime

output = {
    "timestamp": datetime.now().isoformat(),
    "summary": {
        "total": total,
        "success": success_count,
        "disambiguation": disambiguation_count,
        "error": error_count,
        "success_rate": f"{success_count/total*100:.1f}%"
    },
    "by_graph": by_graph,
    "results": all_results
}

with open('test_graph_flows_results.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print("\n✅ Resultados guardados en: test_graph_flows_results.json")